# Reference

https://github.com/khushwant18/Grasp-and-Lift-EEG-detection/blob/master/LSTM.ipynb

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Conv2D, Flatten, MaxPooling2D, Dense,BatchNormalization, Dropout, Input
from tensorflow.keras.layers import LSTM, Dense,BatchNormalization, Dropout, Input
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import matplotlib.pyplot as plt
from sklearn import metrics
import glob
from scipy import signal
from scipy.fft import fft, fftfreq
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn import utils

C:\Users\User\anaconda3\envs\modelop_gpu\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\User\anaconda3\envs\modelop_gpu\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Users\User\anaconda3\envs\modelop_gpu\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:\n%s" %


# Without preprocessing

Training data

In [2]:
#read csv file from the path
xpath = 'grasp-and-lift-eeg-detection/subject1/train/data' 
xfiles = glob.glob(xpath + "/*.csv")
ypath = 'grasp-and-lift-eeg-detection/subject1/train/event' 
yfiles = glob.glob(ypath + "/*.csv")

xli = []
yli = []

for f in xfiles:
    df = pd.read_csv(f, index_col=None, header=0)
    xli.append(df)

for f in yfiles:
    df = pd.read_csv(f, index_col=None, header=0)
    yli.append(df)
    
#concatatenate all the dataframes in the li
x_df = pd.concat(xli, axis=0, ignore_index=True)
x_df = x_df.iloc[:,1:]
x_data=x_df.values

y_df = pd.concat(yli, axis=0, ignore_index=True)
y_df = y_df.iloc[:,1:]
y_data=y_df.values

In [3]:
x_data.shape

(1185498, 32)

Testing data

In [4]:
#read csv file from the path
xtestpath = 'grasp-and-lift-eeg-detection/subject1/test/data' 
xtestfiles = glob.glob(xtestpath + "/*.csv")
ytestpath = 'grasp-and-lift-eeg-detection/subject1/test/event' 
ytestfiles = glob.glob(ytestpath + "/*.csv")

xtestli = []
ytestli = []

for f in xtestfiles:
    df = pd.read_csv(f, index_col=None, header=0)
    xtestli.append(df)

for f in ytestfiles:
    df = pd.read_csv(f, index_col=None, header=0)
    ytestli.append(df)
    
#concatatenate all the dataframes in the li
x_ts_df = pd.concat(xtestli, axis=0, ignore_index=True)
x_ts_df = x_ts_df.iloc[:,1:]
x_ts_data=x_ts_df.values
# x_val_data=x_ts_df.values
# x_val_data=StandardScaler().fit_transform(x_val_data)

y_ts_df = pd.concat(ytestli, axis=0, ignore_index=True)
y_ts_df = y_ts_df.iloc[:,1:]
y_ts_data=y_ts_df.values

In [5]:
x_ts_data.shape

(236894, 32)

Filter [0 0 0 0 0 0] from the data

In [6]:
time_steps=1000
subsample=50

In [7]:
def filter_dataset(x_dataset, y_dataset):
    x_data_filter = []
    y_data_filter = []
    for i in range(x_dataset.shape[0]-time_steps):
        if 1 in y_dataset[i+time_steps]:
            x_data_filter.append(x_dataset[i:i+time_steps:subsample])
            y_data_filter.append(y_dataset[i+time_steps])
    x_data_filter = np.array(x_data_filter)
    y_data_filter = np.array(y_data_filter)
    return x_data_filter, y_data_filter

In [8]:
x_data_filter, y_data_filter = filter_dataset(x_data, y_data)

In [9]:
x_data_filter.shape

(132965, 20, 32)

In [10]:
y_data_filter.shape

(132965, 6)

In [11]:
x_ts_data_filter, y_ts_data_filter = filter_dataset(x_ts_data, y_ts_data)

In [12]:
x_ts_data_filter.shape

(45838, 20, 32)

In [13]:
y_ts_data_filter.shape

(45838, 6)

Shuffle

In [14]:
x_data_filter, y_data_filter = utils.shuffle(x_data_filter, y_data_filter)

In [15]:
x_data_filter.shape

(132965, 20, 32)

In [16]:
y_data_filter.shape

(132965, 6)

Model

In [17]:
def LSTM_model(featr):
    model = Sequential()
    model.add(LSTM(128, return_sequences = True, input_shape = (time_steps//subsample, featr)))
    model.add(Dropout(0.5))
    model.add(LSTM(128, return_sequences = True, input_shape = (time_steps//subsample, featr)))
    model.add(Dropout(0.5))
    model.add(LSTM(128, return_sequences = True, input_shape = (time_steps//subsample, featr)))
    model.add(Dropout(0.5))
    model.add(LSTM(128))
    model.add(Dense(32, activation = "relu"))
    model.add(Dense(6, activation = "sigmoid"))
    
    optimizer = Adam(lr = 0.001)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    model.summary()
    
    return model

In [18]:
model_without_preprocessing = LSTM_model(32)
model_without_preprocessing.fit(x_data_filter, y_data_filter, epochs=30, batch_size=64, validation_split=0.1)

C:\Users\User\anaconda3\envs\modelop_gpu\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 20, 128)           82432     
_________________________________________________________________
dropout (Dropout)            (None, 20, 128)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 20, 128)           131584    
_________________________________________________________________
dropout_1 (Dropout)          (None, 20, 128)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 20, 128)           131584    
_________________________________________________________________
dropout_2 (Dropout)          (None, 20, 128)           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 128)               1

In [19]:
model_without_preprocessing.save("modelop/LSTM/LSTM_without_preprocessing.h5")

Evaluate

In [20]:
model = load_model("modelop/LSTM/LSTM_without_preprocessing.h5")
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 20, 128)           82432     
_________________________________________________________________
dropout (Dropout)            (None, 20, 128)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 20, 128)           131584    
_________________________________________________________________
dropout_1 (Dropout)          (None, 20, 128)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 20, 128)           131584    
_________________________________________________________________
dropout_2 (Dropout)          (None, 20, 128)           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 128)               1

In [21]:
model.evaluate(x_ts_data_filter, y_ts_data_filter)

1433/1433 [==============================] - 16s 10ms/step - loss: 0.5815 - accuracy: 0.6798


[0.5815373063087463, 0.6797634959220886]

# With lowpass filter

Training data with filter

In [18]:
x_data_df=x_df.copy()

In [19]:
fs = 500  # Sampling frequency
fc = 35  # Cut-off frequency of the filter
w = fc / (fs / 2) # Normalize the frequency
for col in x_data_df.columns:
    b, a = signal.butter(8, w, 'low')
    x_data_df[col] = signal.filtfilt(b, a, x_data_df[col].values)

In [20]:
x_data_df.values

array([[ -31.62582713,  368.39071745,  211.47707742, ...,  172.78507556,
         120.51075076,  704.97925546],
       [ -76.07565546,  325.68090395,  188.87485875, ...,  173.26799587,
         113.53506986,  693.9428322 ],
       [-114.68293296,  286.41130889,  169.55635893, ...,  173.40583768,
         106.38561518,  682.60595466],
       ...,
       [ 170.19434836,   43.28088984,   63.64375512, ..., -257.65043159,
        -230.29635042, -278.40120478],
       [ 164.03662925,   16.02791762,   74.66580244, ..., -279.13142422,
        -246.47929981, -299.15333712],
       [ 156.97928734,  -14.68984266,   88.0355826 , ..., -301.69847804,
        -263.87398205, -321.66220475]])

In [21]:
x_data_df.values.shape

(1185498, 32)

Testing data with filter

In [22]:
x_test_df=x_ts_df.copy()

In [23]:
for col in x_test_df.columns:
    b, a = signal.butter(8, w, 'low')
    x_test_df[col] = signal.filtfilt(b, a, x_test_df[col].values)

In [24]:
x_test_df.values

array([[-170.66182563,  135.38743339,  353.32775601, ..., -182.36360743,
         -10.56813655, -112.45719972],
       [-147.8709151 ,  118.38532223,  353.36214175, ..., -176.94995958,
          -5.89847699,  -95.25205859],
       [-126.66368758,  102.67240275,  353.83063837, ..., -172.39239948,
          -1.89969443,  -79.4583339 ],
       ...,
       [ 207.14173229,  347.36445471,  417.56289338, ...,  121.44378851,
         245.99908441,  210.77484768],
       [ 182.79632991,  310.98162743,  411.0147638 , ...,   92.94635083,
         236.42561723,  227.28619971],
       [ 153.07992878,  269.14033758,  402.86840595, ...,   62.01940286,
         225.44384717,  243.24602684]])

In [25]:
x_test_df.values.shape

(236894, 32)

Filter [0 0 0 0 0 0] from the data

In [26]:
x_data_filter, y_data_filter = filter_dataset(x_data_df.values, y_data)

In [27]:
x_data_filter.shape

(132965, 20, 32)

In [28]:
y_data_filter.shape

(132965, 6)

In [29]:
x_ts_data_filter, y_ts_data_filter = filter_dataset(x_test_df.values, y_ts_data)

In [30]:
x_ts_data_filter.shape

(45838, 20, 32)

In [31]:
y_ts_data_filter.shape

(45838, 6)

Shuffle

In [32]:
x_data_filter, y_data_filter = utils.shuffle(x_data_filter, y_data_filter)

In [33]:
x_data_filter.shape

(132965, 20, 32)

In [34]:
y_data_filter.shape

(132965, 6)

Model

In [35]:
model_with_filter = LSTM_model(32)
model_with_filter.fit(x_data_filter, y_data_filter, epochs=30, batch_size=64, validation_split=0.1)

C:\Users\User\anaconda3\envs\modelop_gpu\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 20, 128)           82432     
_________________________________________________________________
dropout (Dropout)            (None, 20, 128)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 20, 128)           131584    
_________________________________________________________________
dropout_1 (Dropout)          (None, 20, 128)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 20, 128)           131584    
_________________________________________________________________
dropout_2 (Dropout)          (None, 20, 128)           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 128)               1

In [36]:
model_with_filter.save("modelop/LSTM/LSTM_with_filter.h5")

Evaluate

In [37]:
model = load_model("modelop/LSTM/LSTM_with_filter.h5")
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 20, 128)           82432     
_________________________________________________________________
dropout (Dropout)            (None, 20, 128)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 20, 128)           131584    
_________________________________________________________________
dropout_1 (Dropout)          (None, 20, 128)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 20, 128)           131584    
_________________________________________________________________
dropout_2 (Dropout)          (None, 20, 128)           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 128)               1

In [38]:
model.evaluate(x_ts_data_filter, y_ts_data_filter)

1433/1433 [==============================] - 10s 7ms/step - loss: 0.7049 - accuracy: 0.6611


[0.7048527002334595, 0.6611108779907227]

# With lowpass filter and standardization

Training data with filter and standardization

In [39]:
Xsc = StandardScaler().fit_transform(x_data_df.values)
Xsc=pd.DataFrame(Xsc)
y=pd.DataFrame(y_data)

In [40]:
Xsc.values

array([[-0.65280336,  0.16159786, -0.85254116, ...,  0.66432352,
         0.73543988,  2.75377775],
       [-0.77843632,  0.01723593, -0.96816935, ...,  0.66700912,
         0.69974879,  2.70147775],
       [-0.8875559 , -0.11549783, -1.06699873, ...,  0.66777568,
         0.66316857,  2.64775394],
       ...,
       [-0.08237898, -0.93729435, -1.60882533, ..., -1.7293968 ,
        -1.05946599, -1.90631912],
       [-0.09978315, -1.02941116, -1.55243886, ..., -1.84885602,
        -1.14226611, -2.00466045],
       [-0.11973002, -1.13323917, -1.48404187, ..., -1.974355  ,
        -1.23126607, -2.1113267 ]])

In [41]:
Xsc.values.shape

(1185498, 32)

Testing data with filter and standardization

In [42]:
x_test_sc=StandardScaler().fit_transform(x_test_df.values)
x_test_sc=pd.DataFrame(x_test_sc)
y_test=pd.DataFrame(y_ts_data)

In [43]:
x_test_sc.values

array([[-5.40640011e-01, -2.18556568e-01,  2.83241317e-01, ...,
        -1.41721093e+00,  2.21421684e-01, -1.07544893e+00],
       [-5.02467038e-01, -3.00638162e-01,  2.83464316e-01, ...,
        -1.37768586e+00,  2.49090880e-01, -9.68511065e-01],
       [-4.66946609e-01, -3.76495887e-01,  2.86502628e-01, ...,
        -1.34441109e+00,  2.72784919e-01, -8.70345805e-01],
       ...,
       [ 9.21510766e-02,  8.04811155e-01,  6.99821341e-01, ...,
         8.00888309e-01,  1.74166284e+00,  9.33586850e-01],
       [ 5.13744525e-02,  6.29164706e-01,  6.57355176e-01, ...,
         5.92828382e-01,  1.68493705e+00,  1.03621249e+00],
       [ 1.60183024e-03,  4.27166276e-01,  6.04524123e-01, ...,
         3.67030589e-01,  1.61986662e+00,  1.13541016e+00]])

In [44]:
x_test_sc.values.shape

(236894, 32)

Filter [0 0 0 0 0 0] from the data

In [45]:
x_data_filter, y_data_filter = filter_dataset(Xsc.values, y_data)

In [46]:
x_data_filter.shape

(132965, 20, 32)

In [47]:
y_data_filter.shape

(132965, 6)

In [48]:
x_ts_data_filter, y_ts_data_filter = filter_dataset(x_test_sc.values, y_ts_data)

In [49]:
x_ts_data_filter.shape

(45838, 20, 32)

In [50]:
y_ts_data_filter.shape

(45838, 6)

Shuffle

In [51]:
x_data_filter, y_data_filter = utils.shuffle(x_data_filter, y_data_filter)

In [52]:
x_data_filter.shape

(132965, 20, 32)

In [53]:
y_data_filter.shape

(132965, 6)

Model

In [54]:
model_with_filter_standard = LSTM_model(32)
model_with_filter_standard.fit(x_data_filter, y_data_filter, epochs=30, batch_size=64, validation_split=0.1)

C:\Users\User\anaconda3\envs\modelop_gpu\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 20, 128)           82432     
_________________________________________________________________
dropout_3 (Dropout)          (None, 20, 128)           0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 20, 128)           131584    
_________________________________________________________________
dropout_4 (Dropout)          (None, 20, 128)           0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 20, 128)           131584    
_________________________________________________________________
dropout_5 (Dropout)          (None, 20, 128)           0         
_________________________________________________________________
lstm_7 (LSTM)                (None, 128)              

In [55]:
model_with_filter_standard.save("modelop/LSTM/LSTM_with_filter_standard.h5")

Evaluate

In [56]:
model = load_model("modelop/LSTM/LSTM_with_filter_standard.h5")
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 20, 128)           82432     
_________________________________________________________________
dropout_3 (Dropout)          (None, 20, 128)           0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 20, 128)           131584    
_________________________________________________________________
dropout_4 (Dropout)          (None, 20, 128)           0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 20, 128)           131584    
_________________________________________________________________
dropout_5 (Dropout)          (None, 20, 128)           0         
_________________________________________________________________
lstm_7 (LSTM)                (None, 128)              

In [57]:
model.evaluate(x_ts_data_filter, y_ts_data_filter)

1433/1433 [==============================] - 12s 7ms/step - loss: 1.1158 - accuracy: 0.6539


[1.1158223152160645, 0.6538679599761963]

# With lowpass filter, PCA and standardization

Training data with filter, PCA and standardization

In [58]:
pca_filter = PCA(n_components = 11)
pca_filter.fit(x_data_df)
x_train_filter_pca = pca_filter.transform(x_data_df)

In [59]:
x_train_filter_pca

array([[  921.05553383,  -442.24009861,  -151.58020016, ...,
         -436.6279772 ,  -257.07678664,   -11.78299407],
       [  898.97738209,  -488.63696708,  -144.26271682, ...,
         -349.41194285,  -243.69923956,     7.33490012],
       [  877.1370104 ,  -528.18506476,  -137.75379474, ...,
         -268.07709435,  -231.65156314,    24.88871284],
       ...,
       [-1284.84105032,   234.35203887,   524.60723009, ...,
           67.80792083,   130.53711506,  -182.49230548],
       [-1321.93204203,   258.6494262 ,   545.79372655, ...,
          124.42249683,   127.84991105,  -175.46293614],
       [-1358.75975879,   282.72673896,   568.68465839, ...,
          184.18083681,   122.56288793,  -168.6919165 ]])

In [60]:
x_train_filter_pca.shape

(1185498, 11)

In [61]:
x_train_filter_pca_st = StandardScaler().fit_transform(x_train_filter_pca)
x_train_filter_pca_st = pd.DataFrame(x_train_filter_pca_st)
y=pd.DataFrame(y_data)

In [62]:
x_train_filter_pca_st.values

array([[ 1.1606786 , -0.86704245, -0.31836441, ..., -2.90264014,
        -1.88790535, -0.10091098],
       [ 1.13285657, -0.95800674, -0.30299548, ..., -2.32284046,
        -1.78966411,  0.06281697],
       [ 1.10533418, -1.03554353, -0.28932477, ..., -1.78213806,
        -1.70118909,  0.21314994],
       ...,
       [-1.61910706,  0.45946346,  1.10183436, ...,  0.45077733,
         0.95863077, -1.56288613],
       [-1.66584769,  0.50710018,  1.14633243, ...,  0.82714291,
         0.93889664, -1.50268576],
       [-1.71225656,  0.55430542,  1.19441033, ...,  1.22440778,
         0.90007011, -1.44469793]])

In [63]:
x_train_filter_pca_st.values.shape

(1185498, 11)

Testing data with filter, PCA and standardization

In [64]:
pca_filter = PCA(n_components = 11)
pca_filter.fit(x_test_df)
x_test_filter_pca = pca_filter.transform(x_test_df)

In [65]:
x_test_filter_pca

array([[-372.49762844,  142.72695169, -122.62956333, ...,  190.78705676,
         128.89918898,   61.88243639],
       [-348.44313597,  115.85373738, -103.79497625, ...,  193.25547153,
         137.17612381,   58.36280675],
       [-326.3533209 ,   92.00084742,  -86.35993601, ...,  194.75792489,
         145.22317763,   55.33152152],
       ...,
       [ 170.19950485, -332.62500492,  -57.60739029, ...,   49.90134532,
          53.50850381,   65.03460981],
       [ 119.82505861, -267.3073893 ,  -34.80502321, ...,   86.65755024,
          73.20797042,   57.24439099],
       [  60.85372164, -194.75930557,   -8.33018609, ...,  126.57748019,
          94.14146229,   48.75905246]])

In [66]:
x_test_filter_pca.shape

(236894, 11)

In [67]:
x_test_filter_pca_st = StandardScaler().fit_transform(x_test_filter_pca)
x_test_filter_pca_st = pd.DataFrame(x_test_filter_pca_st)
y_test=pd.DataFrame(y_ts_data)

In [68]:
x_test_filter_pca_st.values

array([[-0.60183272,  0.26098813, -0.4854349 , ...,  1.54095893,
         1.11062707,  0.60613747],
       [-0.56296863,  0.21184822, -0.4108773 , ...,  1.56089596,
         1.18194317,  0.57166275],
       [-0.52727881,  0.16823122, -0.34185987, ...,  1.57303105,
         1.25127856,  0.54197136],
       ...,
       [ 0.27498599, -0.60823256, -0.22804157, ...,  0.40304581,
         0.46104241,  0.63701295],
       [ 0.19359758, -0.48879385, -0.13777732, ...,  0.69992026,
         0.63077785,  0.56070789],
       [ 0.09831945, -0.35613363, -0.03297543, ...,  1.02234765,
         0.81114596,  0.47759413]])

In [69]:
x_test_filter_pca_st.values.shape

(236894, 11)

Filter [0 0 0 0 0 0] from the data

In [70]:
x_data_filter, y_data_filter = filter_dataset(x_train_filter_pca_st.values, y_data)

In [71]:
x_data_filter.shape

(132965, 20, 11)

In [72]:
y_data_filter.shape

(132965, 6)

In [73]:
x_ts_data_filter, y_ts_data_filter = filter_dataset(x_test_filter_pca_st.values, y_ts_data)

In [74]:
x_ts_data_filter.shape

(45838, 20, 11)

In [75]:
y_ts_data_filter.shape

(45838, 6)

Shuffle

In [76]:
x_data_filter, y_data_filter = utils.shuffle(x_data_filter, y_data_filter)

In [77]:
x_data_filter.shape

(132965, 20, 11)

In [78]:
y_data_filter.shape

(132965, 6)

Model

In [79]:
model_with_filter_standard_PCA = LSTM_model(11)
model_with_filter_standard_PCA.fit(x_data_filter, y_data_filter, epochs=30, batch_size=64, validation_split=0.1)

C:\Users\User\anaconda3\envs\modelop_gpu\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_8 (LSTM)                (None, 20, 128)           71680     
_________________________________________________________________
dropout_6 (Dropout)          (None, 20, 128)           0         
_________________________________________________________________
lstm_9 (LSTM)                (None, 20, 128)           131584    
_________________________________________________________________
dropout_7 (Dropout)          (None, 20, 128)           0         
_________________________________________________________________
lstm_10 (LSTM)               (None, 20, 128)           131584    
_________________________________________________________________
dropout_8 (Dropout)          (None, 20, 128)           0         
_________________________________________________________________
lstm_11 (LSTM)               (None, 128)              

In [80]:
model_with_filter_standard_PCA.save("modelop/LSTM/LSTM_with_filter_PCA_standard11.h5")

Evaluate

In [81]:
model = load_model("modelop/LSTM/LSTM_with_filter_PCA_standard11.h5")
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_8 (LSTM)                (None, 20, 128)           71680     
_________________________________________________________________
dropout_6 (Dropout)          (None, 20, 128)           0         
_________________________________________________________________
lstm_9 (LSTM)                (None, 20, 128)           131584    
_________________________________________________________________
dropout_7 (Dropout)          (None, 20, 128)           0         
_________________________________________________________________
lstm_10 (LSTM)               (None, 20, 128)           131584    
_________________________________________________________________
dropout_8 (Dropout)          (None, 20, 128)           0         
_________________________________________________________________
lstm_11 (LSTM)               (None, 128)              

In [82]:
model.evaluate(x_ts_data_filter, y_ts_data_filter)

1433/1433 [==============================] - 11s 7ms/step - loss: 2.5084 - accuracy: 0.3205


[2.5083794593811035, 0.3205419182777405]

# With lowpass filter 、linear regression and standardization (segment length = 50)

Training data with filter, Linear regression and standardization

In [83]:
def linearregression_seg(signal):
    model = LinearRegression(fit_intercept=True)
    time = np.arange(0, len(signal), 1)
    model.fit(time[:, np.newaxis], signal)
    xfit = np.linspace(time[0], time[len(time)-1], len(signal))
    yfit = model.predict(xfit[:, np.newaxis])
    
    return yfit

def linearregression_allsignal(signal, seg_length):
    new_signal = []
    for i in range((len(signal)//seg_length)+1):
        if (i+1)*seg_length<len(signal):
            new_signal += linearregression_seg(signal[i*seg_length:(i+1)*seg_length]).tolist()
        else:
            new_signal += linearregression_seg(signal[i*seg_length:len(signal)]).tolist()
    
    return np.asarray(new_signal)

In [84]:
x_train_filter_LR = linearregression_allsignal(x_data_df.values, 50)

In [85]:
x_train_filter_LR

array([[ -26.8960533 ,  315.58893394,  255.78336452, ...,  137.40034278,
          42.77381083,  587.34482109],
       [ -23.00706784,  317.99251814,  257.3639416 , ...,  138.33652639,
          43.44477935,  588.09039521],
       [ -19.11808238,  320.39610234,  258.94451867, ...,  139.27271001,
          44.11574788,  588.83596932],
       ...,
       [ 160.97016719,   71.93279005,  105.08121554, ..., -207.07694522,
        -208.20648998, -272.17678639],
       [ 160.25232849,   70.79183253,  102.71171145, ..., -209.24112159,
        -210.22780682, -273.52382141],
       [ 159.53448978,   69.650875  ,  100.34220736, ..., -211.40529796,
        -212.24912367, -274.87085643]])

In [86]:
x_train_filter_LR.shape

(1185498, 32)

In [87]:
x_train_filter_LR_st = StandardScaler().fit_transform(x_train_filter_LR)
x_train_filter_LR_st = pd.DataFrame(x_train_filter_LR_st)
y=pd.DataFrame(y_data)

In [88]:
x_train_filter_LR_st.values

array([[-6.53597198e-01, -1.73361800e-02, -6.50629343e-01, ...,
         4.83335365e-01,  3.47662697e-01,  2.27764694e+00],
       [-6.42361928e-01, -8.99016533e-03, -6.42223694e-01, ...,
         4.88717481e-01,  3.51197005e-01,  2.28131093e+00],
       [-6.31126658e-01, -6.44150701e-04, -6.33818045e-01, ...,
         4.94099596e-01,  3.54731313e-01,  2.28497491e+00],
       ...,
       [-1.10852128e-01, -8.63388396e-01, -1.45207675e+00, ...,
        -1.49706300e+00, -9.74368975e-01, -1.94631417e+00],
       [-1.12925962e-01, -8.67350166e-01, -1.46467799e+00, ...,
        -1.50950484e+00, -9.85016205e-01, -1.95293392e+00],
       [-1.14999796e-01, -8.71311937e-01, -1.47727922e+00, ...,
        -1.52194668e+00, -9.95663435e-01, -1.95955368e+00]])

In [89]:
x_train_filter_LR_st.shape

(1185498, 32)

Testing data with filter, Linear regression and standardization

In [90]:
x_test_filter_LR = linearregression_allsignal(x_test_df.values, 50)

In [91]:
x_test_filter_LR

array([[-130.53744086,   50.69378828,  343.67956619, ..., -193.29943049,
         -22.28747985,  -89.59464607],
       [-128.68764924,   52.00554052,  344.58212035, ..., -192.40500777,
         -20.33619526,  -87.63478345],
       [-126.83785762,   53.31729276,  345.48467452, ..., -191.51058504,
         -18.38491067,  -85.67492082],
       ...,
       [ 134.26335525,  259.16256148,  394.93936179, ...,  161.87268908,
         243.4942507 ,  187.27774941],
       [ 135.68089435,  259.97380917,  397.06115551, ...,  165.04414294,
         246.36244453,  190.50161897],
       [ 137.09843346,  260.78505687,  399.18294923, ...,  168.21559681,
         249.23063836,  193.72548852]])

In [92]:
x_test_filter_LR.shape

(236894, 32)

In [93]:
x_test_filter_LR_st = StandardScaler().fit_transform(x_test_filter_LR)
x_test_filter_LR_st = pd.DataFrame(x_test_filter_LR_st)
y_test=pd.DataFrame(y_ts_data)

In [94]:
x_test_filter_LR_st.values

array([[-0.47528772, -0.64999862,  0.22988428, ..., -1.5827742 ,
         0.15744958, -0.98094809],
       [-0.47217734, -0.64343809,  0.23598195, ..., -1.5758701 ,
         0.16942758, -0.96814539],
       [-0.46906696, -0.63687756,  0.24207962, ..., -1.568966  ,
         0.18140559, -0.95534269],
       ...,
       [-0.03003152,  0.39262588,  0.57619631, ...,  1.15881993,
         1.78895651,  0.82770596],
       [-0.02764796,  0.39668321,  0.59053119, ...,  1.18330056,
         1.80656298,  0.84876572],
       [-0.0252644 ,  0.40074054,  0.60486606, ...,  1.2077812 ,
         1.82416945,  0.86982547]])

In [95]:
x_test_filter_LR_st.shape

(236894, 32)

Filter [0 0 0 0 0 0] from the data

In [96]:
x_data_filter, y_data_filter = filter_dataset(x_train_filter_LR_st.values, y_data)

In [97]:
x_data_filter.shape

(132965, 20, 32)

In [98]:
y_data_filter.shape

(132965, 6)

In [99]:
x_ts_data_filter, y_ts_data_filter = filter_dataset(x_test_filter_LR_st.values, y_ts_data)

In [100]:
x_ts_data_filter.shape

(45838, 20, 32)

In [101]:
y_ts_data_filter.shape

(45838, 6)

Shuffle

In [102]:
x_data_filter, y_data_filter = utils.shuffle(x_data_filter, y_data_filter)

In [103]:
x_data_filter.shape

(132965, 20, 32)

In [104]:
y_data_filter.shape

(132965, 6)

Model

In [105]:
model_with_filter_LG_standard = LSTM_model(32)
model_with_filter_LG_standard.fit(x_data_filter, y_data_filter, epochs=30, batch_size=64, validation_split=0.1)

C:\Users\User\anaconda3\envs\modelop_gpu\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_12 (LSTM)               (None, 20, 128)           82432     
_________________________________________________________________
dropout_9 (Dropout)          (None, 20, 128)           0         
_________________________________________________________________
lstm_13 (LSTM)               (None, 20, 128)           131584    
_________________________________________________________________
dropout_10 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_14 (LSTM)               (None, 20, 128)           131584    
_________________________________________________________________
dropout_11 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_15 (LSTM)               (None, 128)              

In [106]:
model_with_filter_LG_standard.save("modelop/LSTM/LSTM_with_filter_LR50_standard.h5")

Evaluate

In [107]:
model = load_model("modelop/LSTM/LSTM_with_filter_LR50_standard.h5")
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_12 (LSTM)               (None, 20, 128)           82432     
_________________________________________________________________
dropout_9 (Dropout)          (None, 20, 128)           0         
_________________________________________________________________
lstm_13 (LSTM)               (None, 20, 128)           131584    
_________________________________________________________________
dropout_10 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_14 (LSTM)               (None, 20, 128)           131584    
_________________________________________________________________
dropout_11 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_15 (LSTM)               (None, 128)              

In [108]:
model.evaluate(x_ts_data_filter, y_ts_data_filter)

1433/1433 [==============================] - 12s 7ms/step - loss: 1.1085 - accuracy: 0.6956


[1.1084728240966797, 0.6955582499504089]

# With lowpass filter 、linear regression and standardization (segment length = 100)

Training data with filter, Linear regression and standardization

In [109]:
x_train_filter_LR = linearregression_allsignal(x_data_df.values, 100)

In [110]:
x_train_filter_LR

array([[  20.15133089,  345.02321821,  276.15973726, ...,  161.39733714,
          64.31881661,  595.97368359],
       [  21.50066792,  345.87903081,  276.47995694, ...,  161.43100322,
          64.12371138,  596.1433851 ],
       [  22.85000494,  346.73484341,  276.80017662, ...,  161.46466929,
          63.92860616,  596.3130866 ],
       ...,
       [ 171.6830578 ,   91.67288333,  126.99115007, ..., -193.81412869,
        -200.98446484, -277.56176998],
       [ 171.62392001,   91.72248097,  125.92100096, ..., -194.86753611,
        -202.12998954, -278.36388967],
       [ 171.56478222,   91.77207861,  124.85085186, ..., -195.92094353,
        -203.27551423, -279.16600936]])

In [111]:
x_train_filter_LR.shape

(1185498, 32)

In [112]:
x_train_filter_LR_st = StandardScaler().fit_transform(x_train_filter_LR)
x_train_filter_LR_st = pd.DataFrame(x_train_filter_LR_st)
y=pd.DataFrame(y_data)

In [113]:
x_train_filter_LR_st.values

array([[-0.5466633 ,  0.08982776, -0.55705691, ...,  0.63888567,
         0.47219912,  2.37111458],
       [-0.5425468 ,  0.09297304, -0.5553075 , ...,  0.6390847 ,
         0.47114679,  2.3719669 ],
       [-0.5384303 ,  0.09611832, -0.5535581 , ...,  0.63928372,
         0.47009446,  2.37281922],
       ...,
       [-0.08437653, -0.8412851 , -1.37198664, ..., -1.46104494,
        -0.95876065, -2.01619711],
       [-0.08455695, -0.84110282, -1.37783302, ..., -1.46727245,
        -0.96493924, -2.02022573],
       [-0.08473736, -0.84092054, -1.38367941, ..., -1.47349996,
        -0.97111783, -2.02425436]])

In [114]:
x_train_filter_LR_st.shape

(1185498, 32)

Testing data with filter, Linear regression and standardization

In [115]:
x_test_filter_LR = linearregression_allsignal(x_test_df.values, 100)

In [116]:
x_test_filter_LR

array([[ -78.54690609,  112.02318743,  371.73013214, ..., -167.71335711,
          26.90097552,  -37.5413328 ],
       [ -78.8513883 ,  110.75630208,  371.61243801, ..., -168.21814338,
          26.39253849,  -38.20226546],
       [ -79.15587052,  109.48941673,  371.49474387, ..., -168.72292965,
          25.88410146,  -38.86319813],
       ...,
       [ 113.29632415,  250.85313851,  371.10688352, ...,  111.81759164,
         198.99933016,  125.93097676],
       [ 113.50399117,  251.28553765,  372.12821692, ...,  112.44213785,
         199.63026845,  125.88439278],
       [ 113.71165819,  251.71793679,  373.14955032, ...,  113.06668405,
         200.26120674,  125.83780879]])

In [117]:
x_test_filter_LR.shape

(236894, 32)

In [118]:
x_test_filter_LR_st = StandardScaler().fit_transform(x_test_filter_LR)
x_test_filter_LR_st = pd.DataFrame(x_test_filter_LR_st)
y_test=pd.DataFrame(y_ts_data)

In [119]:
x_test_filter_LR_st.values

array([[-0.39213946, -0.35796947,  0.4309562 , ..., -1.45807036,
         0.47422897, -0.66356697],
       [-0.39265708, -0.36457695,  0.43013913, ..., -1.46217159,
         0.47100713, -0.66803709],
       [-0.3931747 , -0.37118442,  0.42932207, ..., -1.46627283,
         0.46778529, -0.6725072 ],
       ...,
       [-0.06600625,  0.36610196,  0.42662944, ...,  0.81303377,
         1.56477322,  0.44205328],
       [-0.06565322,  0.36835715,  0.43371981, ...,  0.81810802,
         1.56877132,  0.44173822],
       [-0.06530019,  0.37061234,  0.44081017, ...,  0.82318227,
         1.57276942,  0.44142315]])

In [120]:
x_test_filter_LR_st.shape

(236894, 32)

Filter [0 0 0 0 0 0] from the data

In [121]:
x_data_filter, y_data_filter = filter_dataset(x_train_filter_LR_st.values, y_data)

In [122]:
x_data_filter.shape

(132965, 20, 32)

In [123]:
y_data_filter.shape

(132965, 6)

In [124]:
x_ts_data_filter, y_ts_data_filter = filter_dataset(x_test_filter_LR_st.values, y_ts_data)

In [125]:
x_ts_data_filter.shape

(45838, 20, 32)

In [126]:
y_ts_data_filter.shape

(45838, 6)

Shuffle

In [127]:
x_data_filter, y_data_filter = utils.shuffle(x_data_filter, y_data_filter)

In [128]:
x_data_filter.shape

(132965, 20, 32)

In [129]:
y_data_filter.shape

(132965, 6)

Model

In [130]:
model_with_filter_LG_standard = LSTM_model(32)
model_with_filter_LG_standard.fit(x_data_filter, y_data_filter, epochs=30, batch_size=64, validation_split=0.1)

C:\Users\User\anaconda3\envs\modelop_gpu\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_16 (LSTM)               (None, 20, 128)           82432     
_________________________________________________________________
dropout_12 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_17 (LSTM)               (None, 20, 128)           131584    
_________________________________________________________________
dropout_13 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_18 (LSTM)               (None, 20, 128)           131584    
_________________________________________________________________
dropout_14 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_19 (LSTM)               (None, 128)              

In [131]:
model_with_filter_LG_standard.save("modelop/LSTM/LSTM_with_filter_LR100_standard.h5")

Evaluate

In [132]:
model = load_model("modelop/LSTM/LSTM_with_filter_LR100_standard.h5")
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_16 (LSTM)               (None, 20, 128)           82432     
_________________________________________________________________
dropout_12 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_17 (LSTM)               (None, 20, 128)           131584    
_________________________________________________________________
dropout_13 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_18 (LSTM)               (None, 20, 128)           131584    
_________________________________________________________________
dropout_14 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_19 (LSTM)               (None, 128)              

In [133]:
model.evaluate(x_ts_data_filter, y_ts_data_filter)

1433/1433 [==============================] - 11s 7ms/step - loss: 1.0237 - accuracy: 0.6677


[1.0236544609069824, 0.6676556468009949]

# With lowpass filter 、linear regression and standardization (segment length = 25)

Training data with filter, Linear regression and standardization

In [134]:
x_train_filter_LR = linearregression_allsignal(x_data_df.values, 25)

In [135]:
x_train_filter_LR

array([[-124.03175324,  302.31524469,  149.58425983, ...,  165.17063472,
         100.18425108,  669.57338561],
       [-111.68075164,  306.24203714,  160.36450083, ...,  163.17850035,
          95.06149978,  662.16880566],
       [ -99.32975004,  310.16882958,  171.14474183, ...,  161.18636599,
          89.93874847,  654.76422571],
       ...,
       [ 166.17866079,   57.5226247 ,   96.59901352, ..., -236.45134008,
        -231.28988721, -282.74257547],
       [ 165.40211913,   54.74575038,   92.84320756, ..., -243.32746196,
        -236.66877604, -286.03433589],
       [ 164.62557748,   51.96887606,   89.08740159, ..., -250.20358385,
        -242.04766487, -289.32609631]])

In [136]:
x_train_filter_LR.shape

(1185498, 32)

In [137]:
x_train_filter_LR_st = StandardScaler().fit_transform(x_train_filter_LR)
x_train_filter_LR_st = pd.DataFrame(x_train_filter_LR_st)
y=pd.DataFrame(y_data)

In [138]:
x_train_filter_LR_st.values

array([[-0.92067856, -0.06241375, -1.19705025, ...,  0.63095218,
         0.63940329,  2.63507019],
       [-0.88551383, -0.04899642, -1.14058579, ...,  0.61971377,
         0.61286213,  2.59931506],
       [-0.8503491 , -0.03557909, -1.08412132, ...,  0.60847535,
         0.58632097,  2.56355993],
       ...,
       [-0.09441592, -0.89883754, -1.47457498, ..., -1.63475575,
        -1.07797604, -1.96345983],
       [-0.09662682, -0.90832575, -1.49424704, ..., -1.67354667,
        -1.10584426, -1.97935504],
       [-0.09883773, -0.91781396, -1.5139191 , ..., -1.71233758,
        -1.13371247, -1.99525025]])

In [139]:
x_train_filter_LR_st.shape

(1185498, 32)

Testing data with filter, Linear regression and standardization

In [140]:
x_test_filter_LR = linearregression_allsignal(x_test_df.values, 25)

In [141]:
x_test_filter_LR

array([[-117.75372654,   75.03014112,  373.44757315, ..., -196.4892563 ,
         -28.14495596,  -91.70435195],
       [-117.35530074,   73.30811413,  371.55695338, ..., -195.21045117,
         -25.72976494,  -89.43241647],
       [-116.95687493,   71.58608715,  369.6663336 , ..., -193.93164604,
         -23.31457391,  -87.160481  ],
       ...,
       [ 175.01099018,  339.41481693,  402.7725556 , ...,  140.89891948,
         254.74090087,  184.79553586],
       [ 183.70633316,  353.63192921,  406.13900018, ...,  138.55591359,
         256.63879581,  187.37046054],
       [ 192.40167615,  367.8490415 ,  409.50544476, ...,  136.21290769,
         258.53669075,  189.94538521]])

In [142]:
x_test_filter_LR.shape

(236894, 32)

In [143]:
x_test_filter_LR_st = StandardScaler().fit_transform(x_test_filter_LR)
x_test_filter_LR_st = pd.DataFrame(x_test_filter_LR_st)
y_test=pd.DataFrame(y_ts_data)

In [144]:
x_test_filter_LR_st.values

array([[-0.45287933, -0.5204245 ,  0.42368563, ..., -1.55725819,
         0.1190356 , -0.96979726],
       [-0.45221074, -0.52890882,  0.41112926, ..., -1.54769493,
         0.13356139, -0.95532795],
       [-0.45154214, -0.53739314,  0.3985729 , ..., -1.53813166,
         0.14808718, -0.94085865],
       ...,
       [ 0.03840724,  0.78218245,  0.61844463, ...,  0.96582586,
         1.8204083 ,  0.79115144],
       [ 0.05299883,  0.85222928,  0.64080254, ...,  0.9483042 ,
         1.83182289,  0.8075504 ],
       [ 0.06759043,  0.92227611,  0.66316045, ...,  0.93078254,
         1.84323749,  0.82394936]])

In [145]:
x_test_filter_LR_st.shape

(236894, 32)

Filter [0 0 0 0 0 0] from the data

In [146]:
x_data_filter, y_data_filter = filter_dataset(x_train_filter_LR_st.values, y_data)

In [147]:
x_data_filter.shape

(132965, 20, 32)

In [148]:
y_data_filter.shape

(132965, 6)

In [149]:
x_ts_data_filter, y_ts_data_filter = filter_dataset(x_test_filter_LR_st.values, y_ts_data)

In [150]:
x_ts_data_filter.shape

(45838, 20, 32)

In [151]:
y_ts_data_filter.shape

(45838, 6)

Shuffle

In [152]:
x_data_filter, y_data_filter = utils.shuffle(x_data_filter, y_data_filter)

In [153]:
x_data_filter.shape

(132965, 20, 32)

In [154]:
y_data_filter.shape

(132965, 6)

Model

In [155]:
model_with_filter_LG_standard = LSTM_model(32)
model_with_filter_LG_standard.fit(x_data_filter, y_data_filter, epochs=30, batch_size=64, validation_split=0.1)

C:\Users\User\anaconda3\envs\modelop_gpu\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_20 (LSTM)               (None, 20, 128)           82432     
_________________________________________________________________
dropout_15 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_21 (LSTM)               (None, 20, 128)           131584    
_________________________________________________________________
dropout_16 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_22 (LSTM)               (None, 20, 128)           131584    
_________________________________________________________________
dropout_17 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_23 (LSTM)               (None, 128)              

In [156]:
model_with_filter_LG_standard.save("modelop/LSTM/LSTM_with_filter_LR25_standard.h5")

Evaluate

In [157]:
model = load_model("modelop/LSTM/LSTM_with_filter_LR25_standard.h5")
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_20 (LSTM)               (None, 20, 128)           82432     
_________________________________________________________________
dropout_15 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_21 (LSTM)               (None, 20, 128)           131584    
_________________________________________________________________
dropout_16 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_22 (LSTM)               (None, 20, 128)           131584    
_________________________________________________________________
dropout_17 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_23 (LSTM)               (None, 128)              

In [158]:
model.evaluate(x_ts_data_filter, y_ts_data_filter)

1433/1433 [==============================] - 12s 8ms/step - loss: 1.0240 - accuracy: 0.6797


[1.0240312814712524, 0.6796544194221497]

# With linear regression and standardization (segment length = 25)

Training data with Linear regression and standardization

In [159]:
x_train_filter_LR = linearregression_allsignal(x_data, 25)

In [160]:
x_train_filter_LR

array([[-139.14153846,  276.70769231,  118.47384615, ...,  152.61230769,
          84.88615385,  658.13230769],
       [-126.03307692,  283.49538462,  131.31769231, ...,  151.65461538,
          80.98230769,  651.87461538],
       [-112.92461538,  290.28307692,  144.16153846, ...,  150.69692308,
          77.07846154,  645.61692308],
       ...,
       [ 162.64822134,   72.71343874,  103.14130435, ..., -230.1284585 ,
        -224.09288538, -272.30434783],
       [ 161.58498024,   71.32411067,  100.32608696, ..., -236.29249012,
        -228.7312253 , -274.82608696],
       [ 160.52173913,   69.93478261,   97.51086957, ..., -242.45652174,
        -233.36956522, -277.34782609]])

In [161]:
x_train_filter_LR.shape

(1185498, 32)

In [162]:
x_train_filter_LR_st = StandardScaler().fit_transform(x_train_filter_LR)
x_train_filter_LR_st = pd.DataFrame(x_train_filter_LR_st)
y=pd.DataFrame(y_data)

In [163]:
x_train_filter_LR_st.values

array([[-0.96331617, -0.14979838, -1.35793673, ...,  0.5597132 ,
         0.55979967,  2.57589548],
       [-0.92600963, -0.12662323, -1.29076557, ...,  0.55431427,
         0.53958612,  2.54572443],
       [-0.88870309, -0.10344809, -1.22359442, ...,  0.54891534,
         0.51937257,  2.51555338],
       ...,
       [-0.10442579, -0.84629372, -1.43812335, ..., -1.59796487,
        -1.04004929, -1.91014323],
       [-0.10745176, -0.85103729, -1.45284646, ..., -1.63271423,
        -1.06406595, -1.92230162],
       [-0.11047773, -0.85578086, -1.46756958, ..., -1.66746359,
        -1.08808261, -1.93446002]])

In [164]:
x_train_filter_LR_st.shape

(1185498, 32)

Testing data with Linear regression and standardization

In [165]:
x_test_filter_LR = linearregression_allsignal(x_ts_data, 25)

In [166]:
x_test_filter_LR

array([[-121.42769231,   72.37846154,  367.81230769, ..., -176.22769231,
         -17.44307692,  -82.66461538],
       [-120.86538462,   70.37692308,  366.40461538, ..., -176.37538462,
         -15.83615385,  -81.13923077],
       [-120.30307692,   68.37538462,  364.99692308, ..., -176.52307692,
         -14.22923077,  -79.61384615],
       ...,
       [ 159.0122807 ,  336.66315789,  384.52105263, ...,  147.65087719,
         257.8       ,  190.70877193],
       [ 165.01403509,  349.54736842,  385.03157895, ...,  145.74385965,
         259.67368421,  193.92280702],
       [ 171.01578947,  362.43157895,  385.54210526, ...,  143.83684211,
         261.54736842,  197.13684211]])

In [167]:
x_test_filter_LR.shape

(236894, 32)

In [168]:
x_test_filter_LR_st = StandardScaler().fit_transform(x_test_filter_LR)
x_test_filter_LR_st = pd.DataFrame(x_test_filter_LR_st)
y_test=pd.DataFrame(y_ts_data)

In [169]:
x_test_filter_LR_st.values

array([[-0.45899459, -0.53253635,  0.38580619, ..., -1.4041699 ,
         0.18328048, -0.91059309],
       [-0.45805108, -0.5423802 ,  0.37646818, ..., -1.40527315,
         0.19293887, -0.90089577],
       [-0.45710758, -0.55222406,  0.36713016, ..., -1.4063764 ,
         0.20259726, -0.89119844],
       ...,
       [ 0.01155917,  0.76725384,  0.49664469, ...,  1.01517696,
         1.8376255 ,  0.82732347],
       [ 0.02162959,  0.83062024,  0.5000313 , ...,  1.00093169,
         1.84888726,  0.84775605],
       [ 0.03170001,  0.89398665,  0.50341791, ...,  0.98668643,
         1.86014901,  0.86818863]])

In [170]:
x_test_filter_LR_st.shape

(236894, 32)

Filter [0 0 0 0 0 0] from the data

In [171]:
x_data_filter, y_data_filter = filter_dataset(x_train_filter_LR_st.values, y_data)

In [172]:
x_data_filter.shape

(132965, 20, 32)

In [173]:
y_data_filter.shape

(132965, 6)

In [174]:
x_ts_data_filter, y_ts_data_filter = filter_dataset(x_test_filter_LR_st.values, y_ts_data)

In [175]:
x_ts_data_filter.shape

(45838, 20, 32)

In [176]:
y_ts_data_filter.shape

(45838, 6)

Shuffle

In [177]:
x_data_filter, y_data_filter = utils.shuffle(x_data_filter, y_data_filter)

In [178]:
x_data_filter.shape

(132965, 20, 32)

In [179]:
y_data_filter.shape

(132965, 6)

Model

In [180]:
model_with_LG_standard = LSTM_model(32)
model_with_LG_standard.fit(x_data_filter, y_data_filter, epochs=30, batch_size=64, validation_split=0.1)

C:\Users\User\anaconda3\envs\modelop_gpu\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_24 (LSTM)               (None, 20, 128)           82432     
_________________________________________________________________
dropout_18 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_25 (LSTM)               (None, 20, 128)           131584    
_________________________________________________________________
dropout_19 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_26 (LSTM)               (None, 20, 128)           131584    
_________________________________________________________________
dropout_20 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_27 (LSTM)               (None, 128)              

In [181]:
model_with_LG_standard.save("modelop/LSTM/LSTM_with_LR25_standard.h5")

Evaluate

In [182]:
model = load_model("modelop/LSTM/LSTM_with_LR25_standard.h5")
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_24 (LSTM)               (None, 20, 128)           82432     
_________________________________________________________________
dropout_18 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_25 (LSTM)               (None, 20, 128)           131584    
_________________________________________________________________
dropout_19 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_26 (LSTM)               (None, 20, 128)           131584    
_________________________________________________________________
dropout_20 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_27 (LSTM)               (None, 128)              

In [183]:
model.evaluate(x_ts_data_filter, y_ts_data_filter)

1433/1433 [==============================] - 12s 7ms/step - loss: 1.0909 - accuracy: 0.6679


[1.0909132957458496, 0.6678737998008728]

# With linear regression and standardization (segment length = 50)

Training data with Linear regression and standardization

In [184]:
x_train_filter_LR = linearregression_allsignal(x_data, 50)

In [185]:
x_train_filter_LR

array([[ -32.08      ,  304.70588235,  235.83764706, ...,  133.62117647,
          37.05882353,  585.42117647],
       [ -28.16653061,  307.66890756,  238.19284514, ...,  134.65133253,
          37.8972389 ,  586.19990396],
       [ -24.25306122,  310.63193277,  240.54804322, ...,  135.6814886 ,
          38.73565426,  586.97863145],
       ...,
       [ 159.4406933 ,   75.04409104,  100.83964756, ..., -203.14922565,
        -202.30981329, -261.09634173],
       [ 158.69568679,   73.87366117,   98.08606528, ..., -205.12903459,
        -204.0248046 , -261.84791576],
       [ 157.95068027,   72.70323129,   95.33248299, ..., -207.10884354,
        -205.73979592, -262.5994898 ]])

In [186]:
x_train_filter_LR.shape

(1185498, 32)

In [187]:
x_train_filter_LR_st = StandardScaler().fit_transform(x_train_filter_LR)
x_train_filter_LR_st = pd.DataFrame(x_train_filter_LR_st)
y=pd.DataFrame(y_data)

In [188]:
x_train_filter_LR_st.values

array([[-0.66849868, -0.05511419, -0.75640357, ...,  0.46151682,
         0.31750334,  2.26727874],
       [-0.65719394, -0.04482777, -0.74388335, ...,  0.467438  ,
         0.32191889,  2.27110411],
       [-0.64588919, -0.03454136, -0.73136313, ...,  0.47335918,
         0.32633444,  2.27492948],
       ...,
       [-0.11525755, -0.85240615, -1.47405208, ..., -1.47418779,
        -0.94314201, -1.89109865],
       [-0.11740963, -0.85646941, -1.4886901 , ..., -1.48556743,
        -0.95217408, -1.89479064],
       [-0.11956172, -0.86053266, -1.50332813, ..., -1.49694706,
        -0.96120616, -1.89848262]])

In [189]:
x_train_filter_LR_st.shape

(1185498, 32)

Testing data with Linear regression and standardization

In [190]:
x_test_filter_LR = linearregression_allsignal(x_ts_data, 50)

In [191]:
x_test_filter_LR

array([[-139.97647059,   46.02823529,  339.86823529, ..., -183.43764706,
         -17.55529412,  -84.72941176],
       [-137.61498199,   47.4630012 ,  340.95116447, ..., -182.87529412,
         -15.76364946,  -82.96494598],
       [-135.2534934 ,   48.89776711,  342.03409364, ..., -182.31294118,
         -13.9720048 ,  -81.20048019],
       ...,
       [ 134.23424947,  267.29915433,  392.08689218, ...,  165.32325581,
         245.75877378,  194.27019027],
       [ 135.82318534,  268.65715292,  394.22071882, ...,  168.57526427,
         248.68696265,  197.93206483],
       [ 137.41212121,  270.01515152,  396.35454545, ...,  171.82727273,
         251.61515152,  201.59393939]])

In [192]:
x_test_filter_LR.shape

(236894, 32)

In [193]:
x_test_filter_LR_st = StandardScaler().fit_transform(x_test_filter_LR)
x_test_filter_LR_st = pd.DataFrame(x_test_filter_LR_st)
y_test=pd.DataFrame(y_ts_data)

In [194]:
x_test_filter_LR_st.values

array([[-0.49115883, -0.67318573,  0.20409822, ..., -1.50631671,
         0.18647417, -0.94879425],
       [-0.48718804, -0.66601153,  0.2114131 , ..., -1.50197685,
         0.19747094, -0.93727252],
       [-0.48321725, -0.65883732,  0.21872798, ..., -1.49763699,
         0.20846771, -0.92575078],
       ...,
       [-0.03007995,  0.4332262 ,  0.55682049, ...,  1.18518241,
         1.80264543,  0.87303699],
       [-0.0274082 ,  0.44001654,  0.57123388, ...,  1.2102792 ,
         1.82061809,  0.89694856],
       [-0.02473644,  0.44680689,  0.58564728, ...,  1.23537598,
         1.83859075,  0.92086012]])

In [195]:
x_test_filter_LR_st.shape

(236894, 32)

Filter [0 0 0 0 0 0] from the data

In [196]:
x_data_filter, y_data_filter = filter_dataset(x_train_filter_LR_st.values, y_data)

In [197]:
x_data_filter.shape

(132965, 20, 32)

In [198]:
y_data_filter.shape

(132965, 6)

In [199]:
x_ts_data_filter, y_ts_data_filter = filter_dataset(x_test_filter_LR_st.values, y_ts_data)

In [200]:
x_ts_data_filter.shape

(45838, 20, 32)

In [201]:
y_ts_data_filter.shape

(45838, 6)

Shuffle

In [202]:
x_data_filter, y_data_filter = utils.shuffle(x_data_filter, y_data_filter)

In [203]:
x_data_filter.shape

(132965, 20, 32)

In [204]:
y_data_filter.shape

(132965, 6)

Model

In [205]:
model_with_LG_standard = LSTM_model(32)
model_with_LG_standard.fit(x_data_filter, y_data_filter, epochs=30, batch_size=64, validation_split=0.1)

C:\Users\User\anaconda3\envs\modelop_gpu\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_28 (LSTM)               (None, 20, 128)           82432     
_________________________________________________________________
dropout_21 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_29 (LSTM)               (None, 20, 128)           131584    
_________________________________________________________________
dropout_22 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_30 (LSTM)               (None, 20, 128)           131584    
_________________________________________________________________
dropout_23 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_31 (LSTM)               (None, 128)              

In [206]:
model_with_LG_standard.save("modelop/LSTM/LSTM_with_LR50_standard.h5")

Evaluate

In [207]:
model = load_model("modelop/LSTM/LSTM_with_LR50_standard.h5")
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_28 (LSTM)               (None, 20, 128)           82432     
_________________________________________________________________
dropout_21 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_29 (LSTM)               (None, 20, 128)           131584    
_________________________________________________________________
dropout_22 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_30 (LSTM)               (None, 20, 128)           131584    
_________________________________________________________________
dropout_23 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_31 (LSTM)               (None, 128)              

In [208]:
model.evaluate(x_ts_data_filter, y_ts_data_filter)

1433/1433 [==============================] - 11s 7ms/step - loss: 1.1609 - accuracy: 0.6579


[1.1608740091323853, 0.657860279083252]

# With linear regression and standardization (segment length = 100)

Training data with Linear regression and standardization

In [209]:
x_train_filter_LR = linearregression_allsignal(x_data, 100)

In [210]:
x_train_filter_LR

array([[  14.09584158,  344.24138614,  269.31425743, ...,  160.06752475,
          62.17544554,  595.72178218],
       [  15.55410741,  345.09428743,  269.72851485, ...,  160.10777678,
          62.00119412,  595.87992799],
       [  17.01237324,  345.94718872,  270.14277228, ...,  160.1480288 ,
          61.82694269,  596.03807381],
       ...,
       [ 169.75592449,   95.35814701,  127.77021211, ..., -191.93141174,
        -199.29141404, -275.07970086],
       [ 169.65192638,   95.46942601,  126.71194588, ..., -192.94529771,
        -200.40915785, -275.84968345],
       [ 169.54792826,   95.58070501,  125.65367965, ..., -193.95918367,
        -201.52690167, -276.61966605]])

In [211]:
x_train_filter_LR.shape

(1185498, 32)

In [212]:
x_train_filter_LR_st = StandardScaler().fit_transform(x_train_filter_LR)
x_train_filter_LR_st = pd.DataFrame(x_train_filter_LR_st)
y=pd.DataFrame(y_data)

In [213]:
x_train_filter_LR_st.values

array([[-0.56510326,  0.08694816, -0.59439189, ...,  0.63099938,
         0.46061807,  2.36964958],
       [-0.56065471,  0.09008252, -0.59212898, ...,  0.63123734,
         0.45967825,  2.3704438 ],
       [-0.55620615,  0.09321688, -0.58986607, ...,  0.63147529,
         0.45873844,  2.37123802],
       ...,
       [-0.09025002, -0.82768335, -1.36758687, ..., -1.44985754,
        -0.94958636, -2.00356234],
       [-0.09056727, -0.82727441, -1.37336773, ..., -1.45585117,
        -0.95561484, -2.00742923],
       [-0.09088452, -0.82686546, -1.37914859, ..., -1.4618448 ,
        -0.96164332, -2.01129613]])

In [214]:
x_train_filter_LR_st.shape

(1185498, 32)

Testing data with Linear regression and standardization

In [215]:
x_test_filter_LR = linearregression_allsignal(x_ts_data, 100)

In [216]:
x_test_filter_LR

array([[ -77.97445545,  112.4170297 ,  372.1160396 , ..., -164.68811881,
          28.92158416,  -34.72514851],
       [ -78.31315332,  111.09708971,  371.97329133, ..., -165.21563156,
          28.3849865 ,  -35.44605461],
       [ -78.65185119,  109.77714971,  371.83054305, ..., -165.74314431,
          27.84838884,  -36.1669607 ],
       ...,
       [ 109.88636347,  251.04984286,  369.17533504, ...,  113.86724705,
         200.39159051,  128.66205252],
       [ 110.01003504,  251.46310732,  370.1809486 , ...,  114.52096955,
         201.04394755,  128.67884261],
       [ 110.13370661,  251.87637178,  371.18656215, ...,  115.17469205,
         201.69630459,  128.6956327 ]])

In [217]:
x_test_filter_LR.shape

(236894, 32)

In [218]:
x_test_filter_LR_st = StandardScaler().fit_transform(x_test_filter_LR)
x_test_filter_LR_st = pd.DataFrame(x_test_filter_LR_st)
y_test=pd.DataFrame(y_ts_data)

In [219]:
x_test_filter_LR_st.values

array([[-0.39116452, -0.3559009 ,  0.43362612, ..., -1.43346938,
         0.48701929, -0.64448885],
       [-0.3917403 , -0.36278482,  0.43263515, ..., -1.43775518,
         0.48361909, -0.64936433],
       [-0.39231609, -0.36966873,  0.43164418, ..., -1.44204098,
         0.48021888, -0.65423982],
       ...,
       [-0.07180243,  0.36711418,  0.41321152, ...,  0.82966467,
         1.57355659,  0.46049803],
       [-0.07159219,  0.36926949,  0.42019257, ...,  0.83497587,
         1.57769032,  0.46061158],
       [-0.07138195,  0.3714248 ,  0.42717362, ...,  0.84028706,
         1.58182405,  0.46072513]])

In [220]:
x_test_filter_LR_st.shape

(236894, 32)

Filter [0 0 0 0 0 0] from the data

In [221]:
x_data_filter, y_data_filter = filter_dataset(x_train_filter_LR_st.values, y_data)

In [222]:
x_data_filter.shape

(132965, 20, 32)

In [223]:
y_data_filter.shape

(132965, 6)

In [224]:
x_ts_data_filter, y_ts_data_filter = filter_dataset(x_test_filter_LR_st.values, y_ts_data)

In [225]:
x_ts_data_filter.shape

(45838, 20, 32)

In [226]:
y_ts_data_filter.shape

(45838, 6)

Shuffle

In [227]:
x_data_filter, y_data_filter = utils.shuffle(x_data_filter, y_data_filter)

In [228]:
x_data_filter.shape

(132965, 20, 32)

In [229]:
y_data_filter.shape

(132965, 6)

Model

In [230]:
model_with_LG_standard = LSTM_model(32)
model_with_LG_standard.fit(x_data_filter, y_data_filter, epochs=30, batch_size=64, validation_split=0.1)

C:\Users\User\anaconda3\envs\modelop_gpu\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_32 (LSTM)               (None, 20, 128)           82432     
_________________________________________________________________
dropout_24 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_33 (LSTM)               (None, 20, 128)           131584    
_________________________________________________________________
dropout_25 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_34 (LSTM)               (None, 20, 128)           131584    
_________________________________________________________________
dropout_26 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_35 (LSTM)               (None, 128)              

In [231]:
model_with_LG_standard.save("modelop/LSTM/LSTM_with_LR100_standard.h5")

Evaluate

In [232]:
model = load_model("modelop/LSTM/LSTM_with_LR100_standard.h5")
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_32 (LSTM)               (None, 20, 128)           82432     
_________________________________________________________________
dropout_24 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_33 (LSTM)               (None, 20, 128)           131584    
_________________________________________________________________
dropout_25 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_34 (LSTM)               (None, 20, 128)           131584    
_________________________________________________________________
dropout_26 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_35 (LSTM)               (None, 128)              

In [233]:
model.evaluate(x_ts_data_filter, y_ts_data_filter)

1433/1433 [==============================] - 12s 7ms/step - loss: 1.1860 - accuracy: 0.6287


[1.18596351146698, 0.6287141442298889]

# With lowpass filter 、Polyregression and standardization (segment length = 100)

Training data with filter, Polyregression and standardization

In [234]:
def polyregression_seg(signal):
    y = signal
    X = np.arange(0, len(signal), 1)
    poly = PolynomialFeatures(degree = 4, include_bias=False)
    X_poly = poly.fit_transform(X.reshape(-1, 1))

    poly.fit(X_poly, y)
    lin2 = LinearRegression()
    lin2.fit(X_poly, y)
    yfit = lin2.predict(X_poly)
    
    return yfit

def polyregression_allsignal(signal, seg_length):
    new_signal = []
    for i in range((len(signal)//seg_length)+1):
        if (i+1)*seg_length<=len(signal):
            new_signal += polyregression_seg(signal[i*seg_length:(i+1)*seg_length]).tolist()
        else:
            new_signal += polyregression_seg(signal[i*seg_length:len(signal)]).tolist()
    
    return np.asarray(new_signal)

In [235]:
x_train_filter_PR = polyregression_allsignal(x_data_df.values, 100)

In [236]:
x_train_filter_PR

array([[-194.63415036,  248.07939244,   76.35780312, ...,  153.02754828,
          92.7647288 ,  659.75294923],
       [-162.98875898,  263.25326336,  108.78140152, ...,  152.99685686,
          87.0360568 ,  651.06726616],
       [-133.50778796,  277.32773351,  138.66922836, ...,  152.88079124,
          81.64764265,  642.96174554],
       ...,
       [ 155.60260168,   33.95537961,   72.07643136, ..., -257.73274037,
        -250.09217788, -308.68859111],
       [ 149.85639555,   22.51537507,   59.18427285, ..., -275.02307928,
        -265.43432204, -324.16776494],
       [ 143.40729388,   10.05973726,   45.17276101, ..., -293.83441988,
        -282.16250738, -341.22145943]])

In [237]:
x_train_filter_PR.shape

(1185498, 32)

In [238]:
x_train_filter_PR_st = StandardScaler().fit_transform(x_train_filter_PR)
x_train_filter_PR_st = pd.DataFrame(x_train_filter_PR_st)
y=pd.DataFrame(y_data)

In [239]:
x_train_filter_PR_st.values

array([[-1.12662345, -0.2491049 , -1.59438656, ...,  0.56827353,
         0.6064662 ,  2.61486383],
       [-1.03612923, -0.19697015, -1.42307713, ...,  0.56809859,
         0.57651393,  2.57248143],
       [-0.95182445, -0.14861274, -1.26516539, ...,  0.56743704,
         0.54834069,  2.53292997],
       ...,
       [-0.12507468, -0.98479743, -1.6170071 , ..., -1.77298714,
        -1.18615582, -2.11071505],
       [-0.14150673, -1.02410328, -1.68512256, ..., -1.87153899,
        -1.26637199, -2.18624677],
       [-0.1599488 , -1.06689867, -1.7591521 , ..., -1.97876029,
        -1.35383505, -2.26946147]])

In [240]:
x_train_filter_PR_st.shape

(1185498, 32)

Testing data with filter, Polyregression and standardization

In [241]:
x_test_filter_PR = polyregression_allsignal(x_test_df.values, 100)

In [242]:
x_test_filter_PR

array([[-131.87744271,   86.02050309,  377.46261801, ..., -191.9785745 ,
         -31.23853061, -100.9945001 ],
       [-129.05681003,   81.20066754,  372.53789844, ..., -190.91300356,
         -27.67197978,  -96.77829444],
       [-126.33840119,   76.94606832,  368.12751089, ..., -189.81987727,
         -24.15432196,  -92.66935851],
       ...,
       [ 169.79552275,  312.35203877,  399.05147957, ...,  146.42609722,
         260.33787849,  192.08395426],
       [ 177.74932264,  323.37681506,  399.16407171, ...,  143.08138938,
         264.46405068,  194.63982029],
       [ 186.12345662,  335.17647771,  398.85744613, ...,  138.88860144,
         268.47561166,  196.90752245]])

In [243]:
x_test_filter_PR.shape

(236894, 32)

In [244]:
x_test_filter_PR_st = StandardScaler().fit_transform(x_test_filter_PR)
x_test_filter_PR_st = pd.DataFrame(x_test_filter_PR_st)
y_test=pd.DataFrame(y_ts_data)

In [245]:
x_test_filter_PR_st.values

array([[-0.47694873, -0.46986943,  0.45502745, ..., -1.55033144,
         0.10155581, -1.04423858],
       [-0.47221179, -0.49379948,  0.42198079, ..., -1.5422226 ,
         0.12324678, -1.01698816],
       [-0.46764652, -0.5149232 ,  0.39238549, ..., -1.53390406,
         0.14464039, -0.99043104],
       ...,
       [ 0.02967812,  0.65384671,  0.59989658, ...,  1.02487994,
         1.87485851,  0.85000306],
       [ 0.04303566,  0.70858376,  0.60065211, ...,  0.99942719,
         1.89995297,  0.86652228],
       [ 0.05709911,  0.76716804,  0.59859454, ...,  0.96752067,
         1.92435039,  0.88117903]])

In [246]:
x_test_filter_PR_st.shape

(236894, 32)

Filter [0 0 0 0 0 0] from the data

In [247]:
x_data_filter, y_data_filter = filter_dataset(x_train_filter_PR_st.values, y_data)

In [248]:
x_data_filter.shape

(132965, 20, 32)

In [249]:
y_data_filter.shape

(132965, 6)

In [250]:
x_ts_data_filter, y_ts_data_filter = filter_dataset(x_test_filter_PR_st.values, y_ts_data)

In [251]:
x_ts_data_filter.shape

(45838, 20, 32)

In [252]:
y_ts_data_filter.shape

(45838, 6)

Shuffle

In [253]:
x_data_filter, y_data_filter = utils.shuffle(x_data_filter, y_data_filter)

In [254]:
x_data_filter.shape

(132965, 20, 32)

In [255]:
y_data_filter.shape

(132965, 6)

Model

In [256]:
model_with_filter_PR_standard = LSTM_model(32)
model_with_filter_PR_standard.fit(x_data_filter, y_data_filter, epochs=30, batch_size=64, validation_split=0.1)

C:\Users\User\anaconda3\envs\modelop_gpu\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_36 (LSTM)               (None, 20, 128)           82432     
_________________________________________________________________
dropout_27 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_37 (LSTM)               (None, 20, 128)           131584    
_________________________________________________________________
dropout_28 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_38 (LSTM)               (None, 20, 128)           131584    
_________________________________________________________________
dropout_29 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_39 (LSTM)               (None, 128)              

In [257]:
model_with_filter_PR_standard.save("modelop/LSTM/LSTM_with_filter_PR100_standard.h5")

Evaluate

In [258]:
model = load_model("modelop/LSTM/LSTM_with_filter_PR100_standard.h5")
model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_36 (LSTM)               (None, 20, 128)           82432     
_________________________________________________________________
dropout_27 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_37 (LSTM)               (None, 20, 128)           131584    
_________________________________________________________________
dropout_28 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_38 (LSTM)               (None, 20, 128)           131584    
_________________________________________________________________
dropout_29 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_39 (LSTM)               (None, 128)              

In [259]:
model.evaluate(x_ts_data_filter, y_ts_data_filter)

1433/1433 [==============================] - 12s 7ms/step - loss: 1.1425 - accuracy: 0.6776


[1.1425107717514038, 0.6776255369186401]

# With lowpass filter 、Polyregression and standardization (segment length = 50)

Training data with filter, Polyregression and standardization

In [260]:
x_train_filter_PR = polyregression_allsignal(x_data_df.values, 50)

In [261]:
x_train_filter_PR

array([[-149.15209453,  243.84408155,  162.25531813, ...,  178.67621442,
         127.94630512,  727.48839081],
       [-130.6052143 ,  266.1716083 ,  164.70281968, ...,  176.81295921,
         118.52016562,  705.91971525],
       [-112.6260032 ,  285.40309868,  169.47535254, ...,  173.87970033,
         108.51542132,  685.27415112],
       ...,
       [ 165.52922745,   40.86729369,   71.26795055, ..., -260.19066997,
        -236.16410412, -285.71606837],
       [ 155.26087392,   16.25215795,   44.43459385, ..., -297.24008868,
        -256.96415749, -307.38397851],
       [ 142.238801  ,  -14.03618528,   12.43476814, ..., -341.28573765,
        -281.83588617, -334.19862031]])

In [262]:
x_train_filter_PR.shape

(1185498, 32)

In [263]:
x_train_filter_PR_st = StandardScaler().fit_transform(x_train_filter_PR)
x_train_filter_PR_st = pd.DataFrame(x_train_filter_PR_st)
y=pd.DataFrame(y_data)

In [264]:
x_train_filter_PR_st.values

array([[-0.98956647, -0.26133262, -1.12412743, ...,  0.70357078,
         0.77974818,  2.89858106],
       [-0.93690146, -0.18529537, -1.11138228, ...,  0.69311252,
         0.73112871,  2.79500756],
       [-0.88584839, -0.11980179, -1.08652975, ...,  0.67664843,
         0.67952483,  2.69586687],
       ...,
       [-0.09600951, -0.9525779 , -1.59793593, ..., -1.75974527,
        -1.09831188, -1.96686061],
       [-0.12516713, -1.03640569, -1.73766821, ..., -1.96769996,
        -1.20559733, -2.07091064],
       [-0.16214411, -1.13955381, -1.90430442, ..., -2.2149238 ,
        -1.33388424, -2.19967544]])

In [265]:
x_train_filter_PR_st.shape

(1185498, 32)

Testing data with filter, Polyregression and standardization

In [266]:
x_test_filter_PR = polyregression_allsignal(x_test_df.values, 50)

In [267]:
x_test_filter_PR

array([[-117.25666166,  178.55345983,  368.85035401, ..., -152.39703946,
          14.15222629,  -86.26703217],
       [-119.55261623,  140.11857353,  365.20158319, ..., -165.77898604,
           3.01304864,  -84.14484423],
       [-121.00787694,  108.01055832,  362.28443203, ..., -176.56031973,
          -5.86828492,  -82.15649428],
       ...,
       [ 184.31743207,  334.96914365,  406.74901342, ...,  121.85463031,
         246.40357925,  200.79291338],
       [ 209.04168347,  356.51002872,  415.92366585, ...,   89.24674356,
         226.01467343,  215.95127952],
       [ 238.23236257,  380.20874005,  427.03436115, ...,   49.33084447,
         199.4949981 ,  233.96764452]])

In [268]:
x_test_filter_PR.shape

(236894, 32)

In [269]:
x_test_filter_PR_st = StandardScaler().fit_transform(x_test_filter_PR)
x_test_filter_PR_st = pd.DataFrame(x_test_filter_PR_st)
y_test=pd.DataFrame(y_ts_data)

In [270]:
x_test_filter_PR_st.values

array([[-0.45179985, -0.01031917,  0.3906199 , ..., -1.21679709,
         0.37131409, -0.9278203 ],
       [-0.45565059, -0.19872987,  0.36654312, ..., -1.31599655,
         0.30469803, -0.91441089],
       [-0.45809132, -0.35612578,  0.347294  , ..., -1.39591783,
         0.25158464, -0.90184716],
       ...,
       [ 0.05399499,  0.75644228,  0.640698  , ...,  0.81621172,
         1.76025595,  0.88601702],
       [ 0.09546208,  0.86203732,  0.70123787, ...,  0.57449169,
         1.63832337,  0.98179774],
       [ 0.14442018,  0.97821018,  0.7745529 , ...,  0.27859787,
         1.47972672,  1.09563722]])

In [271]:
x_test_filter_PR_st.shape

(236894, 32)

Filter [0 0 0 0 0 0] from the data

In [272]:
x_data_filter, y_data_filter = filter_dataset(x_train_filter_PR_st.values, y_data)

In [273]:
x_data_filter.shape

(132965, 20, 32)

In [274]:
y_data_filter.shape

(132965, 6)

In [275]:
x_ts_data_filter, y_ts_data_filter = filter_dataset(x_test_filter_PR_st.values, y_ts_data)

In [276]:
x_ts_data_filter.shape

(45838, 20, 32)

In [277]:
y_ts_data_filter.shape

(45838, 6)

Shuffle

In [278]:
x_data_filter, y_data_filter = utils.shuffle(x_data_filter, y_data_filter)

In [279]:
x_data_filter.shape

(132965, 20, 32)

In [280]:
y_data_filter.shape

(132965, 6)

Model

In [281]:
model_with_filter_PR_standard = LSTM_model(32)
model_with_filter_PR_standard.fit(x_data_filter, y_data_filter, epochs=30, batch_size=64, validation_split=0.1)

C:\Users\User\anaconda3\envs\modelop_gpu\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_40 (LSTM)               (None, 20, 128)           82432     
_________________________________________________________________
dropout_30 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_41 (LSTM)               (None, 20, 128)           131584    
_________________________________________________________________
dropout_31 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_42 (LSTM)               (None, 20, 128)           131584    
_________________________________________________________________
dropout_32 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_43 (LSTM)               (None, 128)             

In [282]:
model_with_filter_PR_standard.save("modelop/LSTM/LSTM_with_filter_PR50_standard.h5")

Evaluate

In [283]:
model = load_model("modelop/LSTM/LSTM_with_filter_PR50_standard.h5")
model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_40 (LSTM)               (None, 20, 128)           82432     
_________________________________________________________________
dropout_30 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_41 (LSTM)               (None, 20, 128)           131584    
_________________________________________________________________
dropout_31 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_42 (LSTM)               (None, 20, 128)           131584    
_________________________________________________________________
dropout_32 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_43 (LSTM)               (None, 128)             

In [284]:
model.evaluate(x_ts_data_filter, y_ts_data_filter)

1433/1433 [==============================] - 12s 7ms/step - loss: 0.9071 - accuracy: 0.6649


[0.9070791006088257, 0.6649068593978882]

# With lowpass filter 、Polyregression and standardization (segment length = 25)

Training data with filter, Polyregression and standardization

In [285]:
x_train_filter_PR = polyregression_allsignal(x_data_df.values, 25)

In [286]:
x_train_filter_PR

array([[  -8.15808281,  402.18043969,  220.83485424, ...,  166.65491726,
         116.73089682,  698.9691818 ],
       [ -87.09519195,  315.86814236,  183.02575721, ...,  175.25747129,
         114.77153479,  695.54641678],
       [-135.42628115,  259.98187132,  160.52877768, ...,  178.47066703,
         109.56293107,  687.40068788],
       ...,
       [ 173.89480203,   47.73587669,   48.82333521, ..., -254.77305307,
        -223.77679339, -267.91301841],
       [ 166.17186658,   20.33014644,   67.20781592, ..., -277.34445446,
        -243.06387083, -293.97510031],
       [ 153.02328959,  -18.33096924,  104.71618207, ..., -304.67577894,
        -271.09101127, -333.55331877]])

In [287]:
x_train_filter_PR.shape

(1185498, 32)

In [288]:
x_train_filter_PR_st = StandardScaler().fit_transform(x_train_filter_PR)
x_train_filter_PR_st = pd.DataFrame(x_train_filter_PR_st)
y=pd.DataFrame(y_data)

In [289]:
x_train_filter_PR_st.values

array([[-0.58664361,  0.27594309, -0.80555237, ...,  0.63058287,
         0.71644902,  2.72790164],
       [-0.80981587, -0.01593956, -0.99918816, ...,  0.67844962,
         0.70641906,  2.71166616],
       [-0.94645829, -0.2049304 , -1.11440435, ...,  0.69632864,
         0.67975624,  2.67302784],
       ...,
       [-0.0719408 , -0.92268356, -1.68649343, ..., -1.71434715,
        -1.02660842, -1.85839163],
       [-0.09377521, -1.01536163, -1.59233902, ..., -1.83994004,
        -1.12533889, -1.98201408],
       [-0.13094907, -1.14610208, -1.40024339, ..., -1.99201833,
        -1.2688097 , -2.16974876]])

In [290]:
x_train_filter_PR_st.shape

(1185498, 32)

Testing data with filter, Polyregression and standardization

In [291]:
x_test_filter_PR = polyregression_allsignal(x_test_df.values, 25)

In [292]:
x_test_filter_PR

array([[-180.45948892,  128.70655977,  354.00972009, ..., -190.49553232,
         -17.47838827, -122.77697117],
       [-145.38919613,  116.44937289,  351.22643641, ..., -174.60078493,
          -3.6634511 ,  -91.58695444],
       [-119.18662029,  105.62730099,  352.11442762, ..., -166.02514663,
           3.67526957,  -70.92767511],
       ...,
       [ 225.08256076,  364.55340139,  424.67774224, ...,  125.57009352,
         247.76076283,  212.49395251],
       [ 194.51098178,  321.98915154,  415.45623862, ...,   95.84514743,
         237.83574266,  228.75014103],
       [ 134.99912163,  251.79379168,  395.68360017, ...,   57.87138621,
         223.68445974,  241.54220433]])

In [293]:
x_test_filter_PR.shape

(236894, 32)

In [294]:
x_test_filter_PR_st = StandardScaler().fit_transform(x_test_filter_PR)
x_test_filter_PR_st = pd.DataFrame(x_test_filter_PR_st)
y_test=pd.DataFrame(y_ts_data)

In [295]:
x_test_filter_PR_st.values

array([[-0.55710671, -0.25107101,  0.28797357, ..., -1.47791674,
         0.180572  , -1.14086042],
       [-0.4983608 , -0.31030697,  0.2699039 , ..., -1.36176423,
         0.26247324, -0.94678419],
       [-0.45446911, -0.36260737,  0.27566892, ..., -1.29909686,
         0.30598053, -0.81823424],
       ...,
       [ 0.12221292,  0.88871856,  0.74676555, ...,  0.83176559,
         1.75303061,  0.94532369],
       [ 0.07100276,  0.68301605,  0.68689756, ...,  0.61454746,
         1.69419057,  1.04647592],
       [-0.028685  ,  0.3437792 ,  0.55852935, ...,  0.33705026,
         1.61029531,  1.12607304]])

In [296]:
x_test_filter_PR_st.shape

(236894, 32)

Filter [0 0 0 0 0 0] from the data

In [297]:
x_data_filter, y_data_filter = filter_dataset(x_train_filter_PR_st.values, y_data)

In [298]:
x_data_filter.shape

(132965, 20, 32)

In [299]:
y_data_filter.shape

(132965, 6)

In [300]:
x_ts_data_filter, y_ts_data_filter = filter_dataset(x_test_filter_PR_st.values, y_ts_data)

In [301]:
x_ts_data_filter.shape

(45838, 20, 32)

In [302]:
y_ts_data_filter.shape

(45838, 6)

Shuffle

In [303]:
x_data_filter, y_data_filter = utils.shuffle(x_data_filter, y_data_filter)

In [304]:
x_data_filter.shape

(132965, 20, 32)

In [305]:
y_data_filter.shape

(132965, 6)

Model

In [306]:
model_with_filter_PR_standard = LSTM_model(32)
model_with_filter_PR_standard.fit(x_data_filter, y_data_filter, epochs=30, batch_size=64, validation_split=0.1)

C:\Users\User\anaconda3\envs\modelop_gpu\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_44 (LSTM)               (None, 20, 128)           82432     
_________________________________________________________________
dropout_33 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_45 (LSTM)               (None, 20, 128)           131584    
_________________________________________________________________
dropout_34 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_46 (LSTM)               (None, 20, 128)           131584    
_________________________________________________________________
dropout_35 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_47 (LSTM)               (None, 128)             

In [307]:
model_with_filter_PR_standard.save("modelop/LSTM/LSTM_with_filter_PR25_standard.h5")

Evaluate

In [308]:
model = load_model("modelop/LSTM/LSTM_with_filter_PR25_standard.h5")
model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_44 (LSTM)               (None, 20, 128)           82432     
_________________________________________________________________
dropout_33 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_45 (LSTM)               (None, 20, 128)           131584    
_________________________________________________________________
dropout_34 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_46 (LSTM)               (None, 20, 128)           131584    
_________________________________________________________________
dropout_35 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_47 (LSTM)               (None, 128)             

In [309]:
model.evaluate(x_ts_data_filter, y_ts_data_filter)

1433/1433 [==============================] - 12s 7ms/step - loss: 1.1459 - accuracy: 0.6321


[1.1459400653839111, 0.6320956349372864]

# With Polyregression and standardization (segment length = 100)

Training data with Polyregression and standardization

In [310]:
x_train_filter_PR = polyregression_allsignal(x_data, 100)

In [311]:
x_train_filter_PR

array([[-217.1886914 ,  235.19179868,   35.00949464, ...,  139.14426238,
          75.21272475,  650.01598845],
       [-183.44793811,  252.22278879,   72.2717891 , ...,  140.93385922,
          71.70523549,  642.79712269],
       [-151.98333276,  267.998665  ,  106.65433306, ...,  142.49525109,
          68.3689552 ,  636.03137103],
       ...,
       [ 152.08107755,   48.55604721,   75.93571694, ..., -253.54786669,
        -243.95987108, -296.37637117],
       [ 146.2101169 ,   39.08963309,   63.58656321, ..., -270.59557403,
        -258.61463096, -310.12834891],
       [ 139.65328014,   28.75452525,   50.15791007, ..., -289.16903172,
        -274.61371322, -325.32175114]])

In [312]:
x_train_filter_PR.shape

(1185498, 32)

In [313]:
x_train_filter_PR_st = StandardScaler().fit_transform(x_train_filter_PR)
x_train_filter_PR_st = pd.DataFrame(x_train_filter_PR_st)
y=pd.DataFrame(y_data)

In [314]:
x_train_filter_PR_st.values

array([[-1.1909424 , -0.29330257, -1.8118115 , ...,  0.48900912,
         0.51457138,  2.56592472],
       [-1.09447066, -0.23480343, -1.61504956, ...,  0.49920675,
         0.49623696,  2.53071933],
       [-1.0045069 , -0.18061543, -1.43349402, ...,  0.508104  ,
         0.47879749,  2.49772371],
       ...,
       [-0.13512435, -0.93437108, -1.59570235, ..., -1.74866107,
        -1.15381366, -2.04949727],
       [-0.15191063, -0.96688693, -1.66091152, ..., -1.8458037 ,
        -1.2304173 , -2.11656372],
       [-0.17065797, -1.00238663, -1.73182094, ..., -1.95164049,
        -1.314048  , -2.1906598 ]])

In [315]:
x_train_filter_PR_st.shape

(1185498, 32)

Testing data with Polyregression and standardization

In [316]:
x_test_filter_PR = polyregression_allsignal(x_ts_data, 100)

In [317]:
x_test_filter_PR

array([[-141.74406082,   68.91580277,  370.38011449, ..., -163.56387837,
         -18.69628414,  -94.08991868],
       [-137.30750116,   66.52046219,  366.5781568 , ..., -166.19867183,
         -16.63137689,  -90.35900813],
       [-133.12030146,   64.48490079,  363.18968022, ..., -168.51791429,
         -14.5070873 ,  -86.72423121],
       ...,
       [ 162.46215748,  319.74685495,  384.62611128, ...,  156.38749057,
         266.23136713,  196.7246605 ],
       [ 169.8702627 ,  332.36680385,  382.12563904, ...,  154.56309608,
         271.14790862,  199.32510123],
       [ 177.70058409,  345.92932605,  378.96449002, ...,  152.01669095,
         276.0082662 ,  201.59682853]])

In [318]:
x_test_filter_PR.shape

(236894, 32)

In [319]:
x_test_filter_PR_st = StandardScaler().fit_transform(x_test_filter_PR)
x_test_filter_PR_st = pd.DataFrame(x_test_filter_PR_st)
y_test=pd.DataFrame(y_ts_data)

In [320]:
x_test_filter_PR_st.values

array([[-0.493495  , -0.55447651,  0.40731532, ..., -1.33355748,
         0.17779387, -0.99904665],
       [-0.48604463, -0.5663624 ,  0.38181461, ..., -1.35359966,
         0.19034944, -0.97494652],
       [-0.47901302, -0.57646303,  0.35908722, ..., -1.37124152,
         0.20326608, -0.95146737],
       ...,
       [ 0.01736223,  0.690169  ,  0.50286692, ...,  1.10022736,
         1.91028268,  0.87949469],
       [ 0.02980276,  0.75279028,  0.4860956 , ...,  1.08634968,
         1.94017747,  0.89629246],
       [ 0.04295231,  0.8200887 ,  0.46489295, ...,  1.06697985,
         1.96973064,  0.91096687]])

In [321]:
x_test_filter_PR_st.shape

(236894, 32)

Filter [0 0 0 0 0 0] from the data

In [322]:
x_data_filter, y_data_filter = filter_dataset(x_train_filter_PR_st.values, y_data)

In [323]:
x_data_filter.shape

(132965, 20, 32)

In [324]:
y_data_filter.shape

(132965, 6)

In [325]:
x_ts_data_filter, y_ts_data_filter = filter_dataset(x_test_filter_PR_st.values, y_ts_data)

In [326]:
x_ts_data_filter.shape

(45838, 20, 32)

In [327]:
y_ts_data_filter.shape

(45838, 6)

Shuffle

In [328]:
x_data_filter, y_data_filter = utils.shuffle(x_data_filter, y_data_filter)

In [329]:
x_data_filter.shape

(132965, 20, 32)

In [330]:
y_data_filter.shape

(132965, 6)

Model

In [331]:
model_with_PR_standard = LSTM_model(32)
model_with_PR_standard.fit(x_data_filter, y_data_filter, epochs=30, batch_size=64, validation_split=0.1)

C:\Users\User\anaconda3\envs\modelop_gpu\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_48 (LSTM)               (None, 20, 128)           82432     
_________________________________________________________________
dropout_36 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_49 (LSTM)               (None, 20, 128)           131584    
_________________________________________________________________
dropout_37 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_50 (LSTM)               (None, 20, 128)           131584    
_________________________________________________________________
dropout_38 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_51 (LSTM)               (None, 128)             

In [332]:
model_with_PR_standard.save("modelop/LSTM/LSTM_with_PR100_standard.h5")

Evaluate

In [333]:
model = load_model("modelop/LSTM/LSTM_with_PR100_standard.h5")
model.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_48 (LSTM)               (None, 20, 128)           82432     
_________________________________________________________________
dropout_36 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_49 (LSTM)               (None, 20, 128)           131584    
_________________________________________________________________
dropout_37 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_50 (LSTM)               (None, 20, 128)           131584    
_________________________________________________________________
dropout_38 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_51 (LSTM)               (None, 128)             

In [334]:
model.evaluate(x_ts_data_filter, y_ts_data_filter)

1433/1433 [==============================] - 11s 7ms/step - loss: 0.8494 - accuracy: 0.6926


[0.8494438529014587, 0.6925694942474365]

# With Polyregression and standardization (segment length = 50)

Training data with Polyregression and standardization

In [335]:
x_train_filter_PR = polyregression_allsignal(x_data, 50)

In [336]:
x_train_filter_PR

array([[-199.68372179,  239.77176357,  104.84797518, ...,  159.17631407,
         105.30727966,  720.22545889],
       [-168.08090325,  259.40858305,  116.85107088, ...,  161.24834822,
          99.62297017,  699.22534041],
       [-139.19670778,  276.86193119,  130.27365408, ...,  161.74673923,
          93.00494327,  679.18858742],
       ...,
       [ 158.93991722,   59.8075555 ,   77.66078287, ..., -253.861813  ,
        -228.41550787, -270.84702495],
       [ 146.38677088,   45.16487397,   56.34475024, ..., -291.04929126,
        -249.79707426, -291.84516352],
       [ 130.69073629,   27.09877643,   31.260004  , ..., -335.55768538,
        -275.77862645, -318.49282367]])

In [337]:
x_train_filter_PR.shape

(1185498, 32)

In [338]:
x_train_filter_PR_st = StandardScaler().fit_transform(x_train_filter_PR)
x_train_filter_PR_st = pd.DataFrame(x_train_filter_PR_st)
y=pd.DataFrame(y_data)

In [339]:
x_train_filter_PR_st.values

array([[-1.13238934, -0.27489391, -1.41980208, ...,  0.59349507,
         0.6623621 ,  2.85722834],
       [-1.04270382, -0.20809463, -1.35744059, ...,  0.60511296,
         0.63306998,  2.7566131 ],
       [-0.96073347, -0.14872295, -1.28770422, ...,  0.60790744,
         0.59896628,  2.66061352],
       ...,
       [-0.11465263, -0.88708465, -1.5610518 , ..., -1.72240858,
        -1.05736244, -1.89117291],
       [-0.15027715, -0.93689519, -1.67179819, ..., -1.93091867,
        -1.16754492, -1.99177866],
       [-0.19482087, -0.99835128, -1.80212475, ..., -2.18047711,
        -1.30143183, -2.11945225]])

In [340]:
x_train_filter_PR_st.shape

(1185498, 32)

Testing data with Polyregression and standardization

In [341]:
x_test_filter_PR = polyregression_allsignal(x_ts_data, 50)

In [342]:
x_test_filter_PR

array([[-114.06463758,  161.79308746,  365.71714619, ..., -108.88806739,
          35.46632232,  -73.76730698],
       [-120.95996547,  125.5116798 ,  361.40974195, ..., -131.74719502,
          19.58045424,  -73.44451   ],
       [-125.78810152,   95.4387374 ,  358.12158251, ..., -150.72026564,
           6.61172555,  -73.14562478],
       ...,
       [ 170.36424827,  340.78781017,  389.59064947, ...,  134.08036834,
         254.16838838,  209.11724056],
       [ 185.66899067,  360.96191195,  387.35204317, ...,  107.75177916,
         237.9821502 ,  226.14249564],
       [ 203.1903155 ,  382.55394136,  384.22922799, ...,   75.55514325,
         216.64784291,  246.1715408 ]])

In [343]:
x_test_filter_PR.shape

(236894, 32)

In [344]:
x_test_filter_PR_st = StandardScaler().fit_transform(x_test_filter_PR)
x_test_filter_PR_st = pd.DataFrame(x_test_filter_PR_st)
y_test=pd.DataFrame(y_ts_data)

In [345]:
x_test_filter_PR_st.values

array([[-0.44636759, -0.09224512,  0.36928548, ..., -0.89275865,
         0.49832262, -0.84673956],
       [-0.45793029, -0.26964913,  0.34091351, ..., -1.06192476,
         0.40340635, -0.84470496],
       [-0.46602654, -0.41669582,  0.3192551 , ..., -1.2023326 ,
         0.32591965, -0.84282108],
       ...,
       [ 0.03058811,  0.78297952,  0.5265352 , ...,  0.90529884,
         1.80504285,  0.93628989],
       [ 0.05625247,  0.88162416,  0.51178998, ...,  0.71045741,
         1.7083319 ,  1.04360056],
       [ 0.08563379,  0.987202  ,  0.49122065, ...,  0.47219024,
         1.58086181,  1.16984421]])

In [346]:
x_test_filter_PR_st.shape

(236894, 32)

Filter [0 0 0 0 0 0] from the data

In [347]:
x_data_filter, y_data_filter = filter_dataset(x_train_filter_PR_st.values, y_data)

In [348]:
x_data_filter.shape

(132965, 20, 32)

In [349]:
y_data_filter.shape

(132965, 6)

In [350]:
x_ts_data_filter, y_ts_data_filter = filter_dataset(x_test_filter_PR_st.values, y_ts_data)

In [351]:
x_ts_data_filter.shape

(45838, 20, 32)

In [352]:
y_ts_data_filter.shape

(45838, 6)

Shuffle

In [353]:
x_data_filter, y_data_filter = utils.shuffle(x_data_filter, y_data_filter)

In [354]:
x_data_filter.shape

(132965, 20, 32)

In [355]:
y_data_filter.shape

(132965, 6)

Model

In [356]:
model_with_PR_standard = LSTM_model(32)
model_with_PR_standard.fit(x_data_filter, y_data_filter, epochs=30, batch_size=64, validation_split=0.1)

C:\Users\User\anaconda3\envs\modelop_gpu\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_52 (LSTM)               (None, 20, 128)           82432     
_________________________________________________________________
dropout_39 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_53 (LSTM)               (None, 20, 128)           131584    
_________________________________________________________________
dropout_40 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_54 (LSTM)               (None, 20, 128)           131584    
_________________________________________________________________
dropout_41 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_55 (LSTM)               (None, 128)             

In [357]:
model_with_PR_standard.save("modelop/LSTM/LSTM_with_PR50_standard.h5")

Evaluate

In [358]:
model = load_model("modelop/LSTM/LSTM_with_PR50_standard.h5")
model.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_52 (LSTM)               (None, 20, 128)           82432     
_________________________________________________________________
dropout_39 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_53 (LSTM)               (None, 20, 128)           131584    
_________________________________________________________________
dropout_40 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_54 (LSTM)               (None, 20, 128)           131584    
_________________________________________________________________
dropout_41 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_55 (LSTM)               (None, 128)             

In [359]:
model.evaluate(x_ts_data_filter, y_ts_data_filter)

1433/1433 [==============================] - 11s 7ms/step - loss: 1.0684 - accuracy: 0.6543


[1.068359613418579, 0.6543478965759277]

# With Polyregression and standardization (segment length = 25)

Training data with Polyregression and standardization

In [360]:
x_train_filter_PR = polyregression_allsignal(x_data, 25)

In [361]:
x_train_filter_PR

array([[ -50.2086649 ,  392.24827586,  176.18785735, ...,  142.46153846,
          83.98956676,  694.49478338],
       [-116.89204244,  303.38924256,  132.15723549, ...,  156.74444444,
          91.16578249,  687.94143826],
       [-156.80410948,  247.53922526,  110.25334897, ...,  165.23675214,
          93.58675662,  678.64066556],
       ...,
       [ 175.74743083,   85.06029188,   45.15053208, ..., -245.26205534,
        -210.73238066, -243.90434783],
       [ 156.52627952,   43.64278909,   79.85305564, ..., -267.29184149,
        -231.36223776, -271.84617412],
       [ 123.85562988,  -25.1122631 ,  143.23210702, ..., -295.803233  ,
        -264.15317726, -319.33534002]])

In [362]:
x_train_filter_PR.shape

(1185498, 32)

In [363]:
x_train_filter_PR_st = StandardScaler().fit_transform(x_train_filter_PR)
x_train_filter_PR_st = pd.DataFrame(x_train_filter_PR_st)
y=pd.DataFrame(y_data)

In [364]:
x_train_filter_PR_st.values

array([[-0.70351937,  0.24107183, -1.02321551, ...,  0.49344563,
         0.5463857 ,  2.67753487],
       [-0.89151059, -0.05783174, -1.24631765, ...,  0.57251557,
         0.58295602,  2.64678453],
       [-1.00402915, -0.24569974, -1.35730414, ...,  0.61952884,
         0.59529342,  2.60314241],
       ...,
       [-0.06651263, -0.79224555, -1.68717861, ..., -1.65298588,
        -0.95553062, -1.72572631],
       [-0.12070016, -0.93156553, -1.51134172, ..., -1.77494242,
        -1.0606613 , -1.85683807],
       [-0.212804  , -1.16284343, -1.19020153, ..., -1.93278102,
        -1.22776541, -2.07967203]])

In [365]:
x_train_filter_PR_st.shape

(1185498, 32)

Testing data with Polyregression and standardization

In [366]:
x_test_filter_PR = polyregression_allsignal(x_ts_data, 25)

In [367]:
x_test_filter_PR

array([[-191.08582375,  103.50433245,  352.91900973, ..., -138.75060419,
           5.60011789, -105.91058061],
       [-154.43082818,  101.20456823,  345.18308282, ..., -135.5290893 ,
          14.97409372,  -78.29991158],
       [-126.41976063,   96.53072438,  343.89759479, ..., -138.38392342,
          17.75148964,  -60.98167455],
       ...,
       [ 229.50172339,  375.39630115,  404.11092892, ...,  134.17170222,
         252.08126221,  213.15616523],
       [ 166.10235074,  320.09479232,  385.76471812, ...,  109.19686568,
         245.68968865,  234.78136052],
       [  56.29893905,  234.16122322,  359.48117329, ...,   80.92677346,
         239.45787393,  260.72852091]])

In [368]:
x_test_filter_PR.shape

(236894, 32)

In [369]:
x_test_filter_PR_st = StandardScaler().fit_transform(x_test_filter_PR)
x_test_filter_PR_st = pd.DataFrame(x_test_filter_PR_st)
y_test=pd.DataFrame(y_ts_data)

In [370]:
x_test_filter_PR_st.values

array([[-0.57437857, -0.36850515,  0.27825736, ..., -1.0907054 ,
         0.31592833, -1.02297433],
       [-0.51303449, -0.37948934,  0.22850566, ..., -1.06735832,
         0.37124566, -0.85331611],
       [-0.46615649, -0.40181266,  0.22023836, ..., -1.08804798,
         0.38763552, -0.7469014 ],
       ...,
       [ 0.12949695,  0.93011187,  0.60748554, ...,  0.88722714,
         1.77045315,  0.93758294],
       [ 0.02339474,  0.66597946,  0.48949642, ...,  0.70622859,
         1.73273545,  1.07046245],
       [-0.16036707,  0.25554146,  0.3204603 , ...,  0.50134854,
         1.69596051,  1.22989898]])

In [371]:
x_test_filter_PR_st.shape

(236894, 32)

Filter [0 0 0 0 0 0] from the data

In [372]:
x_data_filter, y_data_filter = filter_dataset(x_train_filter_PR_st.values, y_data)

In [373]:
x_data_filter.shape

(132965, 20, 32)

In [374]:
y_data_filter.shape

(132965, 6)

In [375]:
x_ts_data_filter, y_ts_data_filter = filter_dataset(x_test_filter_PR_st.values, y_ts_data)

In [376]:
x_ts_data_filter.shape

(45838, 20, 32)

In [377]:
y_ts_data_filter.shape

(45838, 6)

Shuffle

In [378]:
x_data_filter, y_data_filter = utils.shuffle(x_data_filter, y_data_filter)

In [379]:
x_data_filter.shape

(132965, 20, 32)

In [380]:
y_data_filter.shape

(132965, 6)

Model

In [381]:
model_with_PR_standard = LSTM_model(32)
model_with_PR_standard.fit(x_data_filter, y_data_filter, epochs=30, batch_size=64, validation_split=0.1)

C:\Users\User\anaconda3\envs\modelop_gpu\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_56 (LSTM)               (None, 20, 128)           82432     
_________________________________________________________________
dropout_42 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_57 (LSTM)               (None, 20, 128)           131584    
_________________________________________________________________
dropout_43 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_58 (LSTM)               (None, 20, 128)           131584    
_________________________________________________________________
dropout_44 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_59 (LSTM)               (None, 128)             

In [382]:
model_with_PR_standard.save("modelop/LSTM/LSTM_with_PR25_standard.h5")

Evaluate

In [383]:
model = load_model("modelop/LSTM/LSTM_with_PR25_standard.h5")
model.summary()

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_56 (LSTM)               (None, 20, 128)           82432     
_________________________________________________________________
dropout_42 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_57 (LSTM)               (None, 20, 128)           131584    
_________________________________________________________________
dropout_43 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_58 (LSTM)               (None, 20, 128)           131584    
_________________________________________________________________
dropout_44 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_59 (LSTM)               (None, 128)             

In [384]:
model.evaluate(x_ts_data_filter, y_ts_data_filter)

1433/1433 [==============================] - 12s 7ms/step - loss: 1.0191 - accuracy: 0.6218


[1.0190715789794922, 0.621820330619812]